## Carga de librerías

In [ ]:
seed = 161
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Regresion lineal
from sklearn.linear_model import LinearRegression

# Importar/ Exportar modelos
from joblib import dump, load

# Metricas
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error

# Librerías para pipeline
# Composicion de pipelines
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler

## Carga de datos y creación dataframe

In [ ]:
ruta = 'example_data/202220_Laboratorio_3_data_university_admission_train.csv'
df = pd.read_csv(ruta)
print("Número de filas: ", df.shape[0])
print("Número de columnas: ", df.shape[1])
df.sample(5)

In [ ]:
# Rename column 'LOR ' to 'LOR'
df.rename(columns={'LOR ':'LOR'}, inplace=True)

## Pipeline

### Preprocesamiento

In [ ]:
selected_columns = ['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'LOR', 'CGPA', 'Research']

pre_process = [
    ('initial',ColumnTransformer([
        ("selector", 'passthrough',selected_columns), 
        ('imputer', SimpleImputer(strategy='mean'), selected_columns),
        ('binarizer', 'passthrough', ['Research']),
        ('scaler', MinMaxScaler(), ['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'LOR', 'CGPA', ]),
    ])),
]

### Creación del modelo

In [ ]:
# Creación del modelo
model = [('model', LinearRegression())]

### Construcción del Pipeline

In [ ]:
# Create pipe with pre_process, model and export
pipe = Pipeline(pre_process + model)

## Prueba del Pipeline

### Creación de un nuevo modelo

In [ ]:
df.dropna(subset=['Admission Points'], inplace=True)

# Drop rows with null values in Admission Points
df = df.dropna(subset=['Admission Points'])

# Creación de la variable objetivo y de las variables explicativas
y = df['Admission Points']
X = df.drop(['Admission Points'], axis=1)

# Prueba del pipeline con el dataframe df
pipe.fit(X,y)

In [ ]:
y_pred = pipe.predict(X)

# Get the model metrics
print("R^2: ", pipe.score(X,y))
print("MSE: ", mse(y, y_pred))
print("MAPE: ", mean_absolute_percentage_error(y, y_pred)*100, "%")

## Predicción con nuevos datos

In [ ]:
ruta = 'example_data/202220_Laboratorio_3_data_university_admission_test.csv'
df_prueba = pd.read_csv(ruta)
print("Número de filas: ", df.shape[0])
print("Número de columnas: ", df.shape[1])

# Rename column 'LOR ' to 'LOR'
df_prueba.rename(columns={'LOR ':'LOR'}, inplace=True)

df_prueba.sample(5)

In [ ]:
y_pred = pipe.predict(df_prueba)
# Add the predictions to the dataframe
df_prueba['Admission Points'] = y_pred
df_prueba.sample(5)

## Exportar el pipeline

In [ ]:
# Usamos la lbreria joblib
filename = './college_application_api_app/assets/pipe.joblib'
# Se guarda
dump(pipe, filename) 